In [1]:
# Installing necessary libraries

!pip install cellpylib

In [2]:
# Importing necessary libraries

import cellpylib as cpl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
import seaborn as sns
import time
import math
import matplotlib.cm as cm # For colourmap plots later
from tqdm import tqdm

In [3]:
# Parameters

# Data generation parameters
data_size = 100 # the number of data points in each row of data
timesteps = 100 # the number of timesteps which each programme is run for before the output is used to train the model

# Model parameters
num_epochs = 30000  # Number of training epochs
hidden_size = 512  # Update with the desired size of the hidden layer
learning_rate = 0.005 # learning rate used later in the optimizer
batch_size = 128 # Batch size used when creating the train and test datasets. 32 is more suitable for this problem.
epochs = np.arange(0,num_epochs, 1) # Used in plotting
accuracy_frequency = 200 # Used in the evaluation of the model

In [4]:
# Model Initialisation / Training setup

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.bn2 = nn.BatchNorm1d(num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.bn2(out)
        return out

# Define the input size, hidden size, and number of classes
input_size = data_size  # Update with the actual input size
num_classes = 256 #Number of potential classes, here stuck at 256

# Create an instance of the neural network
model = NeuralNetwork(input_size, hidden_size, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
# Define the programme probability distribution, weighting highly autocorrelated programmes more heavily.

def prog_dist_weight_autocorr():
    autocorr_order = [255, 238, 8, 32, 40, 64, 96, 254, 128, 136, 160, 168, 192, 224, 
                      234, 235, 0, 239, 248, 249, 253, 250, 251, 252, 36, 219, 104, 233, 
                      218, 164, 172, 2, 16, 237, 191, 4, 228, 127, 72, 247, 223, 216, 202, 
                      1, 132, 130, 100, 190, 246, 44, 222, 217, 203, 144, 231, 188, 230, 66, 
                      194, 152, 24, 189, 20, 215, 159, 6, 80, 17, 48, 175, 140, 220, 34,
                      119, 207, 12, 206, 95, 196, 221, 245, 3, 243, 68, 63, 10, 146, 187,
                      183, 182, 18, 5, 148, 158, 211, 134, 52, 214, 38, 155, 33, 88, 108, 
                      123, 229, 186, 162, 201, 74, 173, 242, 37, 176, 49, 59, 41, 151, 22, 
                      236, 82, 35, 115, 114, 167, 121, 58, 56, 139, 118, 138, 21, 111, 31, 
                      84, 163, 112, 131, 208, 62, 46, 107, 116, 244, 145, 171, 19, 91, 9, 
                      200, 7, 87, 205, 181, 177, 174, 133, 99, 26, 97, 227, 98, 209, 76, 
                      185, 124, 42, 65, 241, 184, 198, 126, 13, 93, 81, 212, 67, 23, 141,
                      125, 178, 55, 57, 106, 28, 110, 154, 92, 75, 25, 150, 225, 86, 29, 61, 
                      94, 157, 117, 210, 50, 45, 39, 193, 169, 147, 120, 153, 89, 78, 156,
                      137, 15, 161, 73, 70, 103, 43, 204, 11, 122, 60, 85, 179, 170, 102, 30,
                      79, 226, 109, 105, 54, 165, 71, 166, 90, 101, 129, 27, 149, 51, 232, 47, 
                      195, 135, 180, 83, 53, 240, 197, 199, 77, 14, 69, 143, 113, 213, 142]
    programmes_prob_distribution = []
    for i in range(256):
        zipf_weight = (autocorr_order.index(i) + 10)**(-1)
        programmes_prob_distribution.append(zipf_weight)
    prog_prob_dist_norm = [x / sum(programmes_prob_distribution) for x in programmes_prob_distribution]
    return prog_prob_dist_norm

In [6]:
# Sets the above distribution as the probability distribution used throughout the training run

programmes_prob_distribution = prog_dist_weight_autocorr()

In [7]:
# Data generation functions (where the programmes_considered have a probability distribution)

def create_data(data_size, programmes_prob_distribution, number_of_samples, timesteps):

    # Creating the dataset and labels variables to be populated later
    dataset = np.empty(shape=(number_of_samples, data_size), dtype=int) # each row is data_size length, with number_of_samples rows
    labels = np.empty(shape=(1, number_of_samples), dtype=int)

    # Stating the space of considered programmes
    programmes = np.arange(0,256,1)

    # Normalising the distribution in case it is not already normalised
    programmes_total = sum(programmes_prob_distribution)
    programmes_prob_distribution_norm = [x / programmes_total for x in programmes_prob_distribution]
    
    for i in range(number_of_samples):

        # Randomly selecting a rule number according to the probability distribution given
        rule_number = np.random.choice(a = programmes, size=None, replace=True, p = programmes_prob_distribution_norm)
        #print(f"Considering rule_number = ", rule_number)
        cellular_automaton = cpl.init_random(data_size)
        cellular_automaton = cpl.evolve(cellular_automaton, timesteps=timesteps, memoize=True, apply_rule=lambda n, c, t: cpl.nks_rule(n, rule_number))
        #print(cellular_automaton[-1])
        dataset[i] = cellular_automaton[-1]
        labels[:,i] = rule_number

    return [dataset, labels]

def data_loader(data_size, programmes_prob_distribution, number_of_samples, timesteps):

    # Generate the data according to input parameters
    [dataset, labels] = create_data(data_size, programmes_prob_distribution, number_of_samples, timesteps)
    labels = labels[0] # Deal with the fact that the output is a list of a single list

    # Shifting the labels such that they are indexed from 0. Required for cross entropy to work
    #labels = [x - min(labels) for x in labels] #!!! Not currently shifting labels in a test to alter them later - may help with training in smaller batches
    # Use data_split
    data = [(data_sample, label) for data_sample, label in zip(dataset, labels)]

    train_dataset, train_labels = zip(*data)
    
    tensor_train_dataset = TensorDataset(Tensor(train_dataset), Tensor(train_labels))
    
    train_loader = DataLoader(tensor_train_dataset, batch_size=batch_size, shuffle=True)

    return train_loader

In [8]:
# Evaluates a model over the space of all possible functions. Outputs a vector giving that performance

def model_evaluation(model, data_size, timesteps, batch_size):

    # State which programmes are being considered. In this case, it's all of them.
    programmes_considered = np.arange(0,256,1) #NOTE: THIS DISABLING IS TEMPORARY. AIM TO REINTRODUCE IT
    #programmes_considered = np.array([0, 10, 20, 63, 64, 95, 96, 110, 125, 195, 204, 225, 249, 255]) # The programmes used at the moment in autocorrelation

    accuracy_vector = np.empty(256)
    
    for programme in programmes_considered:

        programmes_prob_distribution = [0]*256
        programmes_prob_distribution[programme] = 1

        train_loader = data_loader(data_size, programmes_prob_distribution, batch_size, timesteps)

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for data, labels in train_loader:
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
            accuracy = 100 * correct / total #returns the accuracy as a percentage
            #print(f"For programme " + str(programme) + ": Accuracy = " + str(accuracy))
        accuracy_vector[programme] = accuracy
                
    return accuracy_vector

In [9]:
# Training loop (includes data generation). Note that here training and test loss cease to make much sense

def main_train(data_size, programmes_prob_distribution, batch_size, timesteps, num_epochs):

    # Initisalise training and test loss tracking variables
    training_loss = np.empty(num_epochs)

    # Initialise an array to track not only the general training and test loss, but also the accuracy on individual programme classification during training.
    # This is to attempt to see grokking.
    # Form: Each row of accuracy_array is an epoch, each column of accuracy_array is a binary 1 or 0 based on whether or not it was correctly classified. 
    #accuracy_frequency = 100 # Once every 100 epochs, the accuracy is measured
    accuracy_array = np.empty((math.floor(num_epochs/accuracy_frequency), 256))
    # Initialising variaable for tracking where in the accuracy_array to write data
    eval_count = 0
    # Each epoch here trains over 1 batch size of data (which at the moment is 32). Each epoch is therefore smaller and better controlled.
    for epoch in tqdm(range(num_epochs)):

        # Continually monitoring accuracy of the model by adjusting the accuracy_array
        if epoch%accuracy_frequency==0:
            accuracy_vector = model_evaluation(model, data_size, timesteps, batch_size)
            accuracy_array[eval_count] = accuracy_vector
            eval_count += 1
        
        train_loader = data_loader(data_size, programmes_prob_distribution, batch_size, timesteps)
        for data, labels in train_loader:
            # Forward pass
            outputs = model(data)
    
            #Shifting labels for loss calculation
            shifted_labels = labels - torch.min(labels)
            shifted_labels = shifted_labels.long()
            loss = criterion(outputs, shifted_labels)
                
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Print the loss after each epoch
        if epoch%1000==0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        training_loss[epoch] = loss.item()

    return [training_loss, accuracy_array]

In [10]:
[training_loss, accuracy_array] = main_train(data_size, programmes_prob_distribution, batch_size, timesteps, num_epochs)

  0%|          | 0/30000 [00:00<?, ?it/s]C:\Users\plogo\AppData\Local\Temp\ipykernel_9924\2903999265.py:42: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_6fueooay2f\croot\pytorch-select_1707342446212\work\torch\csrc\utils\tensor_new.cpp:278.)
  tensor_train_dataset = TensorDataset(Tensor(train_dataset), Tensor(train_labels))
  0%|          | 1/30000 [07:35<3792:28:02, 455.11s/it]

Epoch [1/30000], Loss: 5.58183479309082


  2%|▏         | 454/30000 [35:37<38:38:26,  4.71s/it]   


KeyboardInterrupt: 

# Information with present params

For the present parameters as of 10/05/2024, this cell would likely take around 15 hours to run. It would give a very complete display of how programme accuracy changes when all programmes are weighted with autocorrelation. I don't think this plot will be needed, but if it is it is quite easy to generate.

In [ ]:
# Plot accuracy by programme.

rel_epochs = [x for x, index in enumerate(epochs) if index%accuracy_frequency==0]

# Create an axis object for the line plot
fig, ax = plt.subplots()
cmap = cm.get_cmap('rainbow')  # You can choose different colormaps

# Taking values from nearby epochs and averaging
moving_avg = 10 # The size of the averaging window being used.
reshaped_epochs = np.reshape(rel_epochs, (-1, moving_avg))
filtered_epochs = reshaped_epochs[:,0]
repeated_filtered_epochs = np.repeat(filtered_epochs, moving_avg)

for i in tqdm(lines_plotted):
    line = accuracy_array[:,i]
    #color = cmap(lines_plotted[np.where(lines_plotted==i)[0][0]] / (len(lines_plotted) - 1))
    #color = cmap(index / (len(lines_plotted) - 1))
    color = cmap(np.where(lines_plotted == i)[0][0] / (len(lines_plotted) - 1))
    reshaped_line = np.reshape(line, (-1, moving_avg))
    programme_label = 'Programme: ' + str(uncorr_array[np.where(lines_plotted==i)][0])
    pandas_df = pd.DataFrame({programme_label: reshaped_line.flatten(), 'Epochs': repeated_filtered_epochs})
    pandas_df_melted = pd.melt(pandas_df, id_vars = 'Epochs', value_vars = [programme_label], var_name='line', value_name = 'Values')
    #fig, ax = plt.subplots()
    sns.lineplot(data=pandas_df_melted, x='Epochs', y='Values', hue='line', ax=ax, palette = [color])

# Create a ScalarMappable object for the colorbar
norm = plt.Normalize(0, len(lines_plotted) - 1)
sm = cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # Set an empty array to associate with the colorbar
cbar = plt.colorbar(sm, ax=ax)

cbar.set_label('Program Number')

num_ticks = len(lines_plotted)  # Number of desired ticks
indices = np.linspace(0, len(lines_plotted) - 1, num_ticks, dtype=int)
tick_positions = indices
tick_labels = lines_plotted[indices]

cbar.set_ticks(tick_positions)
cbar.set_ticklabels(tick_labels)

ax.get_legend().remove()

plt.ylabel('Accuracy (%)')

plt.show()